In [2]:
# Dependencies 
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import config

In [5]:
## Heather's Code
# Import CSV
fleet_csv = "./Resources/Fleet Data.csv"
fleet_df = pd.read_csv(fleet_csv)

# Dropping unnecessary columns, set the index 
cleaned_fleet = fleet_df.drop(columns=["Aircraft Type","Future","Historic","Total","Orders","Unit Cost","Average Age"]).fillna(value=0)
cleaned_fleet_df = cleaned_fleet.set_index(["Parent Airline"]+["Airline"])

# Remove ',','$' and blank values
cleaned_fleet_df["Total Cost (Current)"] = cleaned_fleet_df["Total Cost (Current)"].str.replace(',', '')
cleaned_fleet_df["Total Cost (Current)"] = cleaned_fleet_df["Total Cost (Current)"].str.replace('$', '')
cleaned_fleet_df = cleaned_fleet_df.dropna()

# Set the objects to integer
cleaned_fleet_df["Total Cost (Current)"] = cleaned_fleet_df["Total Cost (Current)"].astype(str).astype(int)

# Aggregate Columns to Sum
agg_fleet_df = cleaned_fleet_df.groupby(["Parent Airline","Airline"]).agg({"Current":"sum",
                                                                           "Total Cost (Current)":"sum"})

# Change column name to prevent engine errors
agg_fleet_df = agg_fleet_df.reset_index()
agg_fleet_df = agg_fleet_df.rename(columns={"Total Cost (Current)":"total_cost",
                                            "Parent Airline":"parent_airline",
                                            "Airline":"airline",
                                            "Current":"current"})

agg_fleet_df.head()

,parent_airline,airline,current,total_cost
0,Aegean Airlines,Aegean Airlines,47.0,4733
1,Aegean Airlines,Olympic Air,14.0,420
2,Aeroflot,Aeroflot,189.0,22523
3,Aeroflot,Aurora Airlines,20.0,1159
4,Aeroflot,Pobeda,12.0,888


In [4]:
## Michael's code
# Import CSV
airline_csv = "./Resources/Largest Airlines 2015 with Origin or Destination in USA.csv"
airline_df = pd.read_csv(airline_csv)
airline_df.columns

# Clean up the columns
cleaned_airline = airline_df.drop(columns=[" FREIGHT "," MAIL "])
cleaned_airline_df = cleaned_airline.rename(columns={" PASSENGERS ":"passengers",
                                                     " Passenger Miles ":"passenger_miles",
                                                     "CARRIER_NAME":"parent_airline"})
# Remove ','
cleaned_airline_df["passengers"] = cleaned_airline_df["passengers"].str.replace(',', '')
cleaned_airline_df["passenger_miles"] = cleaned_airline_df["passenger_miles"].str.replace(',', '')

# Set the objects to integer
cleaned_airline_df["passengers"] = cleaned_airline_df["passengers"].astype(str).astype(np.int64)
cleaned_airline_df["passenger_miles"] = cleaned_airline_df["passenger_miles"].astype(str).astype(np.int64)

cleaned_airline_df.head()

,passengers,passenger_miles,parent_airline
0,144637180,116627136731,Southwest Airlines Co. Total
1,137916298,186140690281,Delta Air Lines Inc. Total
2,118541411,166869071222,American Airlines Inc. Total
3,95028877,182508547016,United Air Lines Inc. Total
4,35130683,41726258863,JetBlue Airways Total


In [59]:
# Create an engine
connection_string = f'{config.user}:{config.pw}@localhost:5432/airlines_db'
engine = create_engine(f'postgresql://{connection_string}')

In [60]:
# Confirm tables
engine.table_names()

[]

In [65]:
# Cleaning of the data to be able to do a direct inner join in PostgreSQL

# Finding the unique parent airlines
fleet_unique = cleaned_fleet["Parent Airline"].unique()
fleet_unique

airline_unique = cleaned_airline_df["parent_airline"].unique()
airline_unique

# Defining a function to make everything lowercase, and have no spaces
def striplower(s):
    return ''.join(c.lower() for c in s if not c.isspace())

# Creating a dictionary that will include all of the matches found between the two unique lists
# Big shoutout to Sondra for helping us design this loop to keep things relatively simple
airline_rename = {}
for fleet in fleet_unique:
    for airline in airline_unique:
        if striplower(fleet) in striplower(airline):
            airline_rename[airline] = fleet
            
# This is coding used to print what does NOT showup on the rename lists, it is simply used to double check our data for accuracy
for airline in airline_unique:
    airline_found = False
    for key in airline_rename.keys():
        if key == airline:
            airline_found = True
            break
    if not airline_found:
        print(airline)    

US Airways Inc. Total
ExpressJet Airlines Inc. Total
Republic Airlines Total
Envoy Air Total
Endeavor Air Inc. Total
PSA Airlines Inc. Total
Horizon Air Total
Shuttle America Corp. Total
British Airways Plc Total
Compass Airlines Total
GoJet Airlines LLC d/b/a United Express Total
Compagnie Natl Air France Total
Aeromexico Total
Piedmont Airlines Total
Sun Country Airlines d/b/a MN Airlines Total
Compania Panamena (Copa) Total
Transportes Aeros Meridiona Total
Jazz Aviation LP Total
Klm Royal Dutch Airlines Total
Turk Hava Yollari A.O. Total
Aer Lingus Plc Total
Sky Regional Airlines Inc. Total
Swiss International Airlines Total
Aerovias Nacl De Colombia Total
Taca International Airlines Total
Scandinavian Airlines Sys. Total
Caribbean Airlines Limited Total
Iberia Air Lines Of Spain Total
Compagnia Aerea Italiana Total
Icelandair Total
Porter Airlines, Inc. Total
Silver Airways Total
Lan-Chile Airlines Total
Cape Air Total
Aerolitoral Total
Lan Peru Airlines Total
Austrian Airlines To

In [66]:
# Replacing the untidy parent_airline column with the tidy one created with the loop
cleaned_airline_df["parent_airline"] = cleaned_airline_df["parent_airline"].replace(to_replace=airline_rename)

In [67]:
# Connecting to the engines using the final cleaned data
cleaned_airline_df.to_sql(name='airlines', con=engine, if_exists='append', index=False)
agg_fleet_df.to_sql(name='fleet', con=engine, if_exists='append', index=True)